In [7]:
import requests
import json
import socket
import sys
import time


In [8]:
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAJ14TQEAAAAAWFc8hAeE3R9niiRgRjp3QTRgEjc%3DqJmtc23lLR9Fbz3Dp4KbjUHkfpfSUVvNAcjQrkNcmxnCo6BhWv'

In [9]:

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r


In [10]:
def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

In [11]:
def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


In [12]:
def set_rules(delete):
    # You can adjust the rules if needed
    sample_rules = [
            {"value": "Pollution -has:media -has:links -has:images -has:videos  lang: en"},
            {"value": "Science -has:media -has:links -has:images -has:videos  lang: en"},
                 {"value": "Big data -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "#ML -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "earth -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "Money -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "#AI -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "Evolution -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "universe -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "electronics -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "ISRO -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "Education -has:media -has:links -has:images -has:videos  lang: en "},
                 {"value": "#Bigdata -has:media -has:links -has:images -has:videos  lang: en "},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))

In [13]:
def get_stream(set):
    a=100
    b=200
    c=0
    data=[]
    conn = None
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(("localhost", 9876))
    s.listen(1)
    print("Waiting for TCP connection...")
    conn, addr = s.accept()
    #time.sleep(10)
    response = requests.get("https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            a-=1
            json_response = json.loads(response_line)
            #print(json_response)
            data.append(json_response["data"]["text"].replace('\n',' '))
            
            if a<0: 
                conn.send("\n".join(data).encode())
                c+=100
                print(c)
                data=[]
                a=100
                time.sleep(5)
                b-=1
                if b<0:
                    s.shutdown(socket.SHUT_RDWR)
                    s.close()
                    sys.exit()


In [14]:
def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    get_stream(set)

In [15]:
if __name__ == "__main__":
	main()


Exception: Cannot get rules (HTTP 403): {"client_id":"21854365","detail":"When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.","registration_url":"https://developer.twitter.com/en/docs/projects/overview","title":"Client Forbidden","required_enrollment":"Standard Basic","reason":"client-not-enrolled","type":"https://api.twitter.com/2/problems/client-forbidden"}